# Eflux2 applied to *A. niger*

In [1]:
import sys
import pandas as pd
import cobra
sys.path.append('../../src')
from eflux2 import EFlux2

## Read global proteomics normalized to the reference strain SF ABF180_17_R2

[(βAl-3HP)++, pyc2++, Δald6a+NeoR] (Strain ABF-10216R8, ICE ID ABF_011245)

In [12]:
global_prot = pd.read_csv('../../data/round2/normalized_global_proteomics.csv', index_col='Transcript')
global_prot

,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,SF ABF180_4_R1,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
Transcript,,,,,,,,,,,,,,,,,,,,,
1170085,0.967266,0.996633,0.992893,0.988900,1.005546,0.994246,0.995436,0.984235,0.985065,0.987202,...,0.996191,1.000241,0.981752,1.003174,1.000581,1.006389,0.998983,0.974427,0.986005,0.982189
1141495,1.012991,1.010041,1.011290,1.012467,1.008045,1.010177,1.032464,1.009299,1.012114,1.014553,...,1.010923,1.019918,1.003567,1.020978,1.010532,1.022859,1.026123,1.015454,1.016219,1.003530
201546,0.979618,0.980405,0.973220,0.981140,0.998126,0.998079,0.991908,0.981119,0.990187,0.990484,...,0.988765,0.994606,0.979760,0.993260,1.001902,0.996684,0.994010,0.982122,0.986781,0.980817
1147898,0.995988,1.000736,0.999673,1.021090,1.020466,1.023879,1.007351,1.005843,1.003600,1.006999,...,1.007180,1.001447,1.008006,1.004574,1.003261,1.014986,1.022990,1.002505,0.996382,1.002931
1142869,1.018378,1.017542,1.017273,1.015201,1.009343,1.003975,1.016892,1.010228,1.026211,1.010030,...,1.000423,1.002124,1.005939,1.004112,1.001554,1.010091,0.992440,1.000478,0.998578,0.997856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117615,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
210213,1.000000,1.000000,1.000000,1.000000,1.000000,0.926492,0.928163,1.000000,0.951025,1.000000,...,1.013350,1.024441,1.026118,1.015304,1.012428,0.997734,0.990888,0.970595,1.000000,0.954842
1119048,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Read iJB1325 with non-native genes added for 3HP production

In [18]:
# todo: update gurobi license (not required to use gurobi)
model = cobra.io.load_json_model('../../models/iJB1325_HP.nonnative_genes.pubchem.flipped.nonzero.reduced.round2.json')
#model.solver = 'gurobi'
#model.solver

In [19]:
model.summary()

Non-linear or non-reaction model objective. Falling back to minimal display.


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


### No objective function is set for FBA
The biomass reaction is missing in the model. Use CO2 evolution as the objective function since CO2 is the only major organic product not measured

In [21]:
with model:
    model.objective = 'BOUNDARY_CO2e'
    model.reactions.BOUNDARY_GLCe.upper_bound = 1.0
    model.reactions.EX_3hpp_e.lower_bound = 1.0
    model.reactions.BOUNDARY_ETHe.lower_bound = 0.1
    model.reactions.BOUNDARY_EOLe.lower_bound = 0.1
    model.reactions.BOUNDARY_CITe.lower_bound = 0.1
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
GLCe,BOUNDARY_GLCe,1,6,100.00%
O2e,BOUNDARY_O2e,1.264,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
CITe,BOUNDARY_CITe,-0.1,6,10.00%
CO2e,BOUNDARY_CO2e,-1.8,1,30.00%
EOLe,BOUNDARY_EOLe,-0.1,4,6.67%
ETHe,BOUNDARY_ETHe,-0.1,2,3.33%
H2Oe,BOUNDARY_H2Oe,-0.7289,0,0.00%
He,BOUNDARY_He,-1.071,0,0.00%
3hpp_e,EX_3hpp_e,-1,3,50.00%


In [22]:
model.objective = 'BOUNDARY_CO2e'

## Read line rates

In [8]:
#line_rates = pd.read_csv('../../data/round2/Line_rates_include_0hr.csv', index_col='Line Name')
line_rates = pd.read_csv('../../data/round2/Line_rates_exclude_0hr.csv', index_col='Line Name')
line_rates.head()

,Omics Sample ID,Strain (ICE),Genotype,growth_rate,glucose_uptake_rates (mmol/gDCW * hr),3hpacid_secretion_rates (mmol/gDCW * hr),ethanol_uptake_rates (mmol/gDCW * hr),erythritol_secretion_rates (mmol/gDCW * hr),citricacid_uptake_rates (mmol/gDCW * hr)
Line Name,,,,,,,,,
SF ABF180_1_R1,SF ABF180_73,ABF_008340,wild-type,NaN,3.824934,0.000000,0.792616,0.003407,0.013740
SF ABF180_1_R2,SF ABF180_74,ABF_008340,wild-type,NaN,3.557246,0.000000,0.678012,0.012109,0.009979
SF ABF180_1_R3,SF ABF180_75,ABF_008340,wild-type,NaN,3.415469,0.000000,1.080535,0.010234,0.009554
SF ABF180_2_R1,SF ABF180_76,ABF_008343,(βAl-3HP)+,NaN,5.928539,2.040350,0.352739,0.022628,-0.001312
SF ABF180_2_R2,SF ABF180_77,ABF_008343,(βAl-3HP)+,NaN,5.792073,2.099251,0.539265,0.005308,-0.002935


In [9]:
ref_line = 'SF ABF180_17_R2'

ref_line_glucose_rate = line_rates.loc[ref_line,'glucose_uptake_rates (mmol/gDCW * hr)']
ref_line_glucose_rate

6.277123454

## Normalize uptake and secretion rates by glucose uptake rate of reference study line.

In [10]:
normalized_line_rates = line_rates.drop(['Omics Sample ID', 'Strain (ICE)', 'Genotype'], axis=1)
normalized_line_rates = normalized_line_rates.divide(normalized_line_rates.loc[ref_line, 'glucose_uptake_rates (mmol/gDCW * hr)'], axis=1)
normalized_line_rates.to_csv('../../data/round2/normalized_line_rates.csv')
normalized_line_rates

,growth_rate,glucose_uptake_rates (mmol/gDCW * hr),3hpacid_secretion_rates (mmol/gDCW * hr),ethanol_uptake_rates (mmol/gDCW * hr),erythritol_secretion_rates (mmol/gDCW * hr),citricacid_uptake_rates (mmol/gDCW * hr)
Line Name,,,,,,
SF ABF180_1_R1,NaN,0.609345,0.000000,0.126271,0.000543,0.002189
SF ABF180_1_R2,NaN,0.566700,0.000000,0.108013,0.001929,0.001590
SF ABF180_1_R3,NaN,0.544114,0.000000,0.172139,0.001630,0.001522
SF ABF180_2_R1,NaN,0.944467,0.325045,0.056194,0.003605,-0.000209
SF ABF180_2_R2,NaN,0.922727,0.334429,0.085910,0.000846,-0.000467
...,...,...,...,...,...,...
SF ABF180_23_R2,NaN,0.989133,0.478897,0.121175,0.012959,0.001360
SF ABF180_23_R3,NaN,1.137562,0.564211,0.150258,0.013712,0.001255
SF ABF180_24_R1,NaN,1.124896,0.949422,0.043847,0.024140,0.002688


In [23]:
fluxes = {}
for rep in global_prot.columns:
    with model:
        model.reactions.BOUNDARY_GLCe.upper_bound = normalized_line_rates.loc[rep,'glucose_uptake_rates (mmol/gDCW * hr)']
        model.reactions.EX_3hpp_e.lower_bound =  normalized_line_rates.loc[rep,'3hpacid_secretion_rates (mmol/gDCW * hr)']
        model.reactions.BOUNDARY_ETHe.lower_bound = normalized_line_rates.loc[rep,'ethanol_uptake_rates (mmol/gDCW * hr)']
        model.reactions.BOUNDARY_EOLe.lower_bound = normalized_line_rates.loc[rep,'erythritol_secretion_rates (mmol/gDCW * hr)']
        model.reactions.BOUNDARY_CITe.lower_bound = normalized_line_rates.loc[rep,'citricacid_uptake_rates (mmol/gDCW * hr)']
        try:
            print(rep)
            fluxes[rep] = EFlux2(model, global_prot[rep])
        except TypeError:
            print(f"Replicate {rep} with glucose {glucose_uptake} and 3hp {secrete_3hp} is infeasible")

SF ABF180_1_R1
FBA status optimal
FBA solution 2.2325737407985544
EFlux2 status optimal
EFlux2 solution 41.4804943417788
SF ABF180_1_R2
FBA status optimal
FBA solution 2.159653514980294
EFlux2 status optimal
EFlux2 solution 38.56990660722054
SF ABF180_1_R3
FBA status optimal
FBA solution 2.1178277089752995
EFlux2 status optimal
EFlux2 solution 36.93837982872339
SF ABF180_2_R1
FBA status optimal
FBA solution 2.439042887671277
EFlux2 status optimal
EFlux2 solution 48.52589042368618
SF ABF180_2_R2
FBA status optimal
FBA solution 2.399970928221042
EFlux2 status optimal
EFlux2 solution 47.459524096679615
SF ABF180_2_R3
FBA status optimal
FBA solution 2.511128897786185
EFlux2 status optimal
EFlux2 solution 50.62443966085311
SF ABF180_3_R1
FBA status optimal
FBA solution 2.2548188107572638
EFlux2 status optimal
EFlux2 solution 46.57734663769634
SF ABF180_3_R2
FBA status optimal
FBA solution 2.3132027865874782
EFlux2 status optimal
EFlux2 solution 47.442901891055044
SF ABF180_3_R3
FBA status o

/Users/kimj972/anaconda3/envs/python3_cobrapy/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


FBA status infeasible
FBA solution 0.28067313373714
EFlux2 status infeasible
EFlux2 solution 10.39335410404209
SF ABF180_5_R2
FBA status infeasible
FBA solution 0.0759268548986452
EFlux2 status infeasible
EFlux2 solution 10.966126091484313
SF ABF180_5_R3
FBA status infeasible
FBA solution 0.6505052096787267
EFlux2 status infeasible
EFlux2 solution 43.493608625327994
SF ABF180_6_R1
FBA status optimal
FBA solution 2.005967529783064
EFlux2 status optimal
EFlux2 solution 37.04289658131873
SF ABF180_6_R2
FBA status optimal
FBA solution 2.209780551113635
EFlux2 status optimal
EFlux2 solution 44.795607699019456
SF ABF180_6_R3
FBA status optimal
FBA solution 2.25813195274616
EFlux2 status optimal
EFlux2 solution 45.112578543254585
SF ABF180_7_R1
FBA status optimal
FBA solution 1.8773668078422945
EFlux2 status optimal
EFlux2 solution 39.111585976133625
SF ABF180_7_R2
FBA status optimal
FBA solution 1.9205052551668844
EFlux2 status optimal
EFlux2 solution 45.69651890746901
SF ABF180_7_R3
FBA sta

In [13]:
for rep in global_prot.columns:
    if fluxes[rep].status is not 'optimal':
        print(rep, fluxes[rep].status)

SF ABF180_5_R1 infeasible
SF ABF180_5_R2 infeasible
SF ABF180_5_R3 infeasible
SF ABF180_17_R1 infeasible
SF ABF180_17_R2 infeasible
SF ABF180_17_R3 infeasible
SF ABF180_18_R1 infeasible
SF ABF180_19_R1 infeasible
SF ABF180_19_R3 infeasible
SF ABF180_20_R1 infeasible
SF ABF180_20_R2 infeasible
SF ABF180_21_R1 infeasible
SF ABF180_21_R2 infeasible
SF ABF180_21_R3 infeasible
SF ABF180_22_R1 infeasible
SF ABF180_22_R2 infeasible
SF ABF180_22_R3 infeasible


In [59]:
import numpy as np
from optlang.symbolics import add
from eflux2 import get_eflux2_bounds
from eflux.eflux2 import add_slack_variables_to_model

In [70]:
# get eflux2 solutions by adding slack variables for infeasible cases
for rep in global_prot.columns:
    if fluxes[rep].status is not 'optimal':
        print(rep)
        # get eflux2 bounds from global proteomics
        eflux2_bounds = get_eflux2_bounds(model, global_prot[rep])
        upper_bounds = {k: v[1] if v[1] <= 1000.0 else 1000.0 for k, v in eflux2_bounds.items()}
        # build a relaxed model with slack variables
        relaxed_model = add_slack_variables_to_model(model, upper_bounds)
        # solve FBA to calculate the maximum CO2
        relaxed_model.tolerance = 1e-9
        fba_sol = relaxed_model.optimize()
        print('FBA status', fba_sol.status)
        print('FBA solution', fba_sol.objective_value)
        # Constrain the CO2 secretion to the optimal value
        for r in relaxed_model.reactions:
            if r.objective_coefficient == 1.0:
                r.lower_bound = fba_sol[r.id]
        # minimize the sum of squared flux values
        relaxed_model.objective = relaxed_model.problem.Objective(add([r.flux_expression**2 for r in relaxed_model.reactions]), direction='min')
        eflux2_sol = relaxed_model.optimize()
        print('EFlux2 status', eflux2_sol.status)
        print('EFlux2 solution', eflux2_sol.objective_value)
        fluxes[rep] = eflux2_sol

SF ABF180_5_R1
FBA status optimal
FBA solution 3.266083158889731
EFlux2 status optimal
EFlux2 solution 65.10123329038805
SF ABF180_5_R2
FBA status optimal
FBA solution 3.2402123266446576
EFlux2 status optimal
EFlux2 solution 64.07397673266576
SF ABF180_5_R3
FBA status optimal
FBA solution 3.2636475168093284
EFlux2 status optimal
EFlux2 solution 65.00417257745289
SF ABF180_17_R1
FBA status optimal
FBA solution 3.2868716788271204
EFlux2 status optimal
EFlux2 solution 65.932605464528
SF ABF180_17_R2
FBA status optimal
FBA solution 3.304347826086983
EFlux2 status optimal
EFlux2 solution 66.63559071014704
SF ABF180_17_R3
FBA status optimal
FBA solution 3.29151605778759
EFlux2 status optimal
EFlux2 solution 66.11906382283513
SF ABF180_18_R1
FBA status optimal
FBA solution 3.29199771594676
EFlux2 status optimal
EFlux2 solution 66.13841607032799
SF ABF180_19_R1
FBA status optimal
FBA solution 3.2990853401265996
EFlux2 status optimal
EFlux2 solution 66.42351277986995
SF ABF180_19_R3
FBA status 

In [71]:
for rep in global_prot.columns:
    if fluxes[rep].status is not 'optimal':
        print(rep, fluxes[rep].status)

In [74]:
efluxes = pd.DataFrame(dict([(rep, fluxes[rep].to_frame()['fluxes']) for rep in fluxes]))
eflux_rates = efluxes*ref_line_glucose_rate
eflux_rates.to_csv('../../data/round2/Eflux2_flux_rates.csv')